# TabM

This is a standalone usage example for the TabM project.
The easiest way to run it is [Pixi](https://pixi.sh/latest/#installation):

```shell
git clone https://github.com/yandex-research/tabm
cd tabm

# With GPU:
pixi run -e cuda jupyter-lab example.ipynb

# Without GPU:
pixi run jupyter-lab example.ipynb
```

For the full overview of the project, and for non-Pixi environment setups, see README in the repository:
https://github.com/yandex-research/tabm

In [1]:
!git clone https://github.com/yandex-research/tabm
%cd tabm

Cloning into 'tabm'...
remote: Enumerating objects: 50606, done.
remote: Counting objects: 100% (12277/12277), done.
remote: Compressing objects: 100% (4503/4503), done.
remote: Total 50606 (delta 7843), reused 12201 (delta 7773), pack-reused 38329 (from 1)
Receiving objects: 100% (50606/50606), 12.13 MiB | 21.41 MiB/s, done.
Resolving deltas: 100% (30991/30991), done.
Updating files: 100% (37498/37498), done.
/kaggle/working/tabm


In [2]:
!pip install torch

In [3]:
# !pip install torcheval

In [4]:
!pip install rtdl_num_embeddings>=0.0.11

In [5]:
from tabm_reference import Model

In [7]:
# ruff: noqa: E402
import math
import random
import warnings
from typing import Literal, NamedTuple

import numpy as np
import rtdl_num_embeddings  # https://github.com/yandex-research/rtdl-num-embeddings
import scipy.special
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing
import torch
import torch.nn.functional as F
import torch.optim
from torch import Tensor
from tqdm.std import tqdm

warnings.simplefilter('ignore')
from tabm_reference import Model, make_parameter_groups

warnings.resetwarnings()

In [8]:
seed = 0
random.seed(seed)
np.random.seed(seed + 1)
torch.manual_seed(seed + 2)
pass

# Dataset

In [9]:
import pandas as pd

# Load the uploaded data files
inputs_path = '/kaggle/input/heart-disease-prediction-llm-hackathon/inputs.csv'
labels_path = '/kaggle/input/heart-disease-prediction-llm-hackathon/labels.csv'

inputs_df = pd.read_csv(inputs_path)
labels_df = pd.read_csv(labels_path)

data = inputs_df.merge(labels_df, on="PatientID")

# Separate features and target
X = data.drop(columns=["PatientID", "HadHeartAttack"])
y = data["HadHeartAttack"]


# Display the first few rows of the data to understand its structure

binary_columns = [
    'Sex', 'HadAngina', 'HadStroke',
    'HadAsthma', 'HadSkinCancer', 'HadCOPD', 'HadDepressiveDisorder',
    'HadKidneyDisease', 'HadArthritis', 'DeafOrHardOfHearing', 'BlindOrVisionDifficulty',
    'DifficultyConcentrating', 'DifficultyWalking',
    'DifficultyDressingBathing', 'DifficultyErrands', 'ChestScan', 'AlcoholDrinkers',
    'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver', 'HighRiskLastYear', 'CovidPos'
]

for col in binary_columns:
    assert len(X[col].unique()) == 2

cat_columns = [
    'State', 'GeneralHealth', 'AgeCategory',
    'HadDiabetes', 'SmokerStatus', 'ECigaretteUsage',
    'RaceEthnicityCategory', 'TetanusLast10Tdap'
]

for col in cat_columns:
    print(col, len(X[col].unique()))

num_columns = ['HeightInMeters', 'WeightInKilograms', 'BMI']

for col in num_columns:
    print(col, X[col].min(), X[col].max())

X_cont = X.loc[:, num_columns].to_numpy().astype(np.float32)
X_cat = X.loc[:, binary_columns + cat_columns]

for col in binary_columns + cat_columns:
    X_cat[col], _ = pd.factorize(X_cat[col])

X_cat = X_cat.to_numpy()
Y, _ = pd.factorize(y)
Y = Y.astype(np.int64)

task_type = "classification"

State 54
GeneralHealth 5
AgeCategory 13
HadDiabetes 4
SmokerStatus 4
ECigaretteUsage 4
RaceEthnicityCategory 5
TetanusLast10Tdap 4
HeightInMeters 0.910000026226044 2.41000008583069
WeightInKilograms 28.1200008392334 292.570007324219
BMI 12.0200004577637 97.6500015258789


In [10]:
X_cat.max(axis=0)

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, 53,  4, 12,  3,  3,  3,  4,  3])

In [11]:
cat_cardinalities = [x + 1 for x in X_cat.max(axis=0)]
n_cont_features = X_cont.shape[1]
n_classes = 2

In [13]:
print(f"2's: {sum(x == 2 for x in cat_cardinalities)}")
print(f"3's: {sum(x == 4 for x in cat_cardinalities)}")
print(f"4's: {sum(x == 5 for x in cat_cardinalities)}")

2's: 22
3's: 4
4's: 2


In [14]:
Y.max(), Y.shape

(1, (190157,))

In [15]:
all_idx = np.arange(len(Y))
trainval_idx, test_idx = sklearn.model_selection.train_test_split(
    all_idx, train_size=0.9
)
train_idx, val_idx = sklearn.model_selection.train_test_split(
    trainval_idx, train_size=0.9
)
data_numpy = {
    'train': {'x_cont': X_cont[train_idx], 'y': Y[train_idx]},
    'val': {'x_cont': X_cont[val_idx], 'y': Y[val_idx]},
    'test': {'x_cont': X_cont[test_idx], 'y': Y[test_idx]},
}
if X_cat is not None:
    data_numpy['train']['x_cat'] = X_cat[train_idx]
    data_numpy['val']['x_cat'] = X_cat[val_idx]
    data_numpy['test']['x_cat'] = X_cat[test_idx]

In [16]:
data_numpy['train']['x_cont'].shape

(154026, 3)

# Data preprocessing

In [17]:
# np.random.rand?

In [18]:
# Feature preprocessing.
# NOTE
# The choice between preprocessing strategies depends on a task and a model.

# Simple preprocessing strategy.
preprocessing = sklearn.preprocessing.StandardScaler().fit(
    data_numpy['train']['x_cont']
)

for part in data_numpy:
    data_numpy[part]['x_cont'] = preprocessing.transform(data_numpy[part]['x_cont'])

# print(data_numpy['train']['x_cont'].mean(), data_numpy['train']['x_cont'].std())
# Advanced preprocessing strategy.
# The noise is added to improve the output of QuantileTransformer in some cases.
X_cont_train_numpy = data_numpy['train']['x_cont']
noise = (
    np.random.default_rng(0)
    .normal(0.0, 1e-1, X_cont_train_numpy.shape)
    .astype(X_cont_train_numpy.dtype)
)
preprocessing = sklearn.preprocessing.QuantileTransformer(
    n_quantiles=max(min(len(train_idx) // 30, 1000), 10),
    output_distribution='normal',
    subsample=10**9,
).fit(X_cont_train_numpy + noise)
del X_cont_train_numpy

# Apply the preprocessing.
for part in data_numpy:
    data_numpy[part]['x_cont'] = preprocessing.transform(data_numpy[part]['x_cont'])

BINARY_AUGMENT_P = 0.
X_cat_numpy = data_numpy['train']['x_cat']
for col in range(len(binary_columns)):
    k = int(BINARY_AUGMENT_P * X_cat_numpy.shape[0])
    indices = np.random.choice(X_cat_numpy.shape[0], size=k, replace=False)
    X_cat_numpy[indices, col] = 1 - X_cat_numpy[indices, col]

CAT_AUGMENT_P = 0.1
for col in range(len(binary_columns), len(binary_columns) + len(cat_columns)):
    k = int(CAT_AUGMENT_P * X_cat_numpy.shape[0])
    indices = np.random.choice(X_cat_numpy.shape[0], size=k, replace=False)
    random_categories = np.random.choice(X_cat_numpy[:, col].max(axis=0) + 1, size=k)
    X_cat_numpy[indices, col] = random_categories

data_numpy['train']['x_cat'] = X_cat_numpy


# Label preprocessing.
class RegressionLabelStats(NamedTuple):
    mean: float
    std: float


Y_train = data_numpy['train']['y'].copy()
if task_type == 'regression':
    # For regression tasks, it is highly recommended to standardize the training labels.
    regression_label_stats = RegressionLabelStats(
        Y_train.mean().item(), Y_train.std().item()
    )
    Y_train = (Y_train - regression_label_stats.mean) / regression_label_stats.std
else:
    regression_label_stats = None

In [19]:
data_numpy['train']['y'].shape

(154026,)

In [20]:
# # balance the train dataset

# print('prev train shape:', data_numpy['train']['y'].shape[0])

# class_balance = 1.5

# pos_indices = np.nonzero(data_numpy['train']['y'] == 1)[0]
# neg_indices = np.nonzero(data_numpy['train']['y'] == 0)[0]

# print(pos_indices.shape[0], neg_indices.shape[0])

# new_neg_indices = np.random.choice(neg_indices.shape[0], size=int(class_balance * pos_indices.shape[0]), replace=False)
# print(new_neg_indices.shape[0])
# for key in data_numpy['train']:
#     new_data = np.concatenate([data_numpy['train'][key][pos_indices], data_numpy['train'][key][new_neg_indices]], axis=0)
#     print(new_data.shape)
#     shuffled_indices = np.random.permutation(new_data.shape[0])
#     data_numpy['train'][key] = new_data[shuffled_indices]

# print('new train shape:', data_numpy['train']['y'].shape[0])

#  PyTorch settings

In [21]:
# Device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Convert data to tensors
data = {
    part: {k: torch.as_tensor(v, device=device) for k, v in data_numpy[part].items()}
    for part in data_numpy
}
Y_train = torch.as_tensor(Y_train, device=device)
if task_type == 'regression':
    for part in data:
        data[part]['y'] = data[part]['y'].float()
    Y_train = Y_train.float()

# Automatic mixed precision (AMP)
# torch.float16 is implemented for completeness,
# but it was not tested in the project,
# so torch.bfloat16 is used by default.
amp_dtype = (
    torch.bfloat16
    if torch.cuda.is_available() and torch.cuda.is_bf16_supported()
    else torch.float16
    if torch.cuda.is_available()
    else None
)
# Changing False to True will result in faster training on compatible hardware.
amp_enabled = False and amp_dtype is not None
grad_scaler = torch.cuda.amp.GradScaler() if amp_dtype is torch.float16 else None  # type: ignore

# torch.compile
compile_model = False

# fmt: off
print(
    f'Device:        {device.type.upper()}'
    f'\nAMP:           {amp_enabled} (dtype: {amp_dtype})'
    f'\ntorch.compile: {compile_model}'
)
# fmt: on

Device:        CUDA
AMP:           False (dtype: torch.bfloat16)
torch.compile: False


# Model

In [61]:
# Choose one of the two configurations below.
from torch.optim.swa_utils import AveragedModel, SWALR, update_bn

# TabM
arch_type = 'tabm'
bins = None
k = 32

# TabM-mini with the piecewise-linear embeddings.
# arch_type = 'tabm-mini'
# bins = rtdl_num_embeddings.compute_bins(data['train']['x_cont'])

model = Model(
    n_num_features=n_cont_features,
    cat_cardinalities=cat_cardinalities,
    n_classes=n_classes,
    backbone={
        'type': 'MLP',
        'n_blocks': 3 if bins is None else 2,
        'd_block': 512,
        'dropout': 0.1,
    },
    bins=bins,
    num_embeddings=(
        None
        if bins is None
        else {
            'type': 'PiecewiseLinearEmbeddings',
            'd_embedding': 16,
            'activation': False,
            'version': 'B',
        }
    ),
    arch_type=arch_type,
    k=k,
).to(device)
swa_model = AveragedModel(model)

optimizer = torch.optim.AdamW(make_parameter_groups(model), lr=2e-3, weight_decay=3e-4) #was 3e-4
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200, eta_min=1e-5)
swa_scheduler = SWALR(optimizer, swa_lr=1.8e-3)

if compile_model:
    # NOTE
    # `torch.compile` is intentionally called without the `mode` argument
    # (mode="reduce-overhead" caused issues during training with torch==2.0.1).
    model = torch.compile(model)
    evaluation_mode = torch.no_grad
else:
    evaluation_mode = torch.inference_mode

In [62]:
pos_weight = (len(Y_train) - Y_train.sum()) / Y_train.sum()
pos_weight

tensor(16.9517, device='cuda:0')

In [63]:
(len(Y_train) - Y_train.sum()) / Y_train.sum()

tensor(16.9517, device='cuda:0')

In [64]:
(len(data['val']['y']) - data['val']['y'].sum()) / data['val']['y'].sum()

tensor(17.5831, device='cuda:0')

In [65]:
(len(data['test']['y']) - data['test']['y'].sum()) / data['test']['y'].sum()

tensor(16.7554, device='cuda:0')

In [66]:
# from torcheval.metrics.functional import binary_f1_score

def find_best_threshold(y_true, y_prob, metric=sklearn.metrics.f1_score):
    """
    Find the best threshold for a binary classification task based on a given metric.

    Parameters:
    - y_true: array-like, shape (n_samples,)
      True binary labels (0 or 1).
    - y_prob: array-like, shape (n_samples,)
      Predicted probabilities for the positive class.
    - metric: callable
      The metric to optimize. Default is F1-score.

    Returns:
    - best_threshold: float
      The threshold that maximizes the specified metric.
    - best_score: float
      The best metric score.
    """
    best_threshold = 0.0
    best_score = 0.0

    # Search thresholds from 0.0 to 1.0 with small steps
    thresholds = np.linspace(0, 1, 25)

    for threshold in thresholds:
        # Convert probabilities to binary predictions
        y_pred = (y_prob >= threshold).astype(int)

        # Compute the metric
        score = metric(y_true, y_pred)

        # Update best threshold if the current score is better
        if score > best_score:
            best_score = score
            best_threshold = threshold

    return best_threshold, best_score

In [67]:
@torch.autocast(device.type, enabled=amp_enabled, dtype=amp_dtype)  # type: ignore[code]
def apply_model(model, part: str, idx: Tensor) -> Tensor:
    pred = model(
        data[part]['x_cont'][idx],
        data[part]['x_cat'][idx] if 'x_cat' in data[part] else None,
    )
    # print(pred.shape, pred[..., 0].shape)
    if task_type != "regression":
        return pred.float()

    return (
        pred
        .squeeze(-1)  # Remove the last dimension for regression tasks.
        .float()
    )


if task_type == "regression":
    base_loss_fn = F.mse_loss
elif n_classes == 2:
    base_loss_fn = F.binary_cross_entropy_with_logits # lambda x, y: F.binary_cross_entropy_with_logits(x, y, reduction='none')
else:
    base_loss_fn = F.cross_entropy

def focal_loss(predictions, targets, alpha=0.25, gamma=2.0):
    bce_loss = F.binary_cross_entropy_with_logits(predictions, targets, reduction='none', pos_weight=torch.tensor(2.))
    pt = torch.exp(-bce_loss)  # Probability of correct class
    loss = alpha * (1 - pt)**gamma * bce_loss
    return loss.mean()

def loss_fn(y_pred: Tensor, y_true: Tensor) -> Tensor:
    # TabM produces k predictions per object. Each of them must be trained separately.
    # (regression)     y_pred.shape == (batch_size, k)
    # (classification) y_pred.shape == (batch_size, k, n_classes)
    if task_type == 'regression':
        k = y_pred.shape[-1]
    else:
        k = y_pred.shape[-2]

    y_pred = y_pred[..., 1]
    # print(y_pred.shape)

    # return base_loss_fn(y_pred.flatten(0, 1), y_true.repeat_interleave(k))
    # print(k, y_pred.shape, y_pred.flatten(0, 1).shape, y_true.repeat_interleave(k).shape)
    return focal_loss(y_pred.flatten(0, 1), y_true.repeat_interleave(k)) #torch.tensor(10).to(y_pred.device)
    # return base_loss_fn(
    #     y_pred.flatten(0, 1), y_true.repeat_interleave(k),
    #     pos_weight=pos_weight #torch.tensor(10).to(y_pred.device)
    # )


@evaluation_mode()
def evaluate(model, part: str) -> float:
    model.eval()

    # When using torch.compile, you may need to reduce the evaluation batch size.
    eval_batch_size = 8096
    y_pred: np.ndarray = (
        torch.cat(
            [
                apply_model(model, part, idx)
                for idx in torch.arange(len(data[part]['y']), device=device).split(
                    eval_batch_size
                )
            ]
        )
        .cpu()
        .numpy()
    )
    if task_type == 'regression':
        # Transform the predictions back to the original label space.
        assert regression_label_stats is not None
        y_pred = y_pred * regression_label_stats.std + regression_label_stats.mean

    # print(y_pred.shape)
    # Compute the mean of the k predictions.
    if task_type != 'regression':
        # For classification, the mean must be computed in the probabily space.
        y_pred = scipy.special.softmax(y_pred, axis=-1)
    # print(y_pred.shape)
    y_pred = y_pred.mean(1)
    # print(y_pred.shape)
    print('mean prediction:', (y_pred[..., -1] > 0.5).mean())

    y_true = data[part]['y'].cpu().numpy()

    # print(y_pred.shape)
    # print(y_true.shape)

    if task_type == 'regression':
        score = -(sklearn.metrics.mean_squared_error(y_true, y_pred) ** 0.5)
    else:
        y_pred_final = y_pred[..., -1] > 0.5
        score = sklearn.metrics.accuracy_score(y_true, y_pred_final)
        recall = sklearn.metrics.recall_score(y_true, y_pred_final)
        precision = sklearn.metrics.precision_score(y_true, y_pred_final)
        print(f'({part}) recall: {recall:.4f}')
        print(f'({part}) precision: {precision:.4f}')

        best_threshold, best_score = find_best_threshold(
            y_true, y_pred[..., -1], metric=sklearn.metrics.f1_score
        )

        print(f'({part}) BEST threshold: {best_threshold:.4f}')
        print(f'({part}) BEST f1: {best_score:.4f}')

        recall = sklearn.metrics.recall_score(y_true, (y_pred[..., -1] >= best_threshold).astype(int))
        precision = sklearn.metrics.precision_score(y_true, (y_pred[..., -1] >= best_threshold).astype(int))
        print(f'({part}) BEST recall: {recall:.4f}')
        print(f'({part}) BEST precision: {precision:.4f}')

    return float(score)  # The higher -- the better.


print(f'Test score before training: {evaluate(model, "test"):.4f}')

mean prediction: 1.0
(test) recall: 1.0000
(test) precision: 0.0563
(test) BEST threshold: 0.0000
(test) BEST f1: 0.1066
(test) BEST recall: 1.0000
(test) BEST precision: 0.0563
Test score before training: 0.0563


In [68]:
task_type

'classification'

# Training

In [69]:
def print_grad_norms(model):
    total_norm = 0.0
    for p in model.parameters():
        if p.grad is not None:
            param_norm = p.grad.data.norm(2)  # L2 norm of the gradient
            total_norm += param_norm.item() ** 2  # Accumulate squared norms

    total_norm = total_norm ** 0.5  # Final L2 norm across all parameters
    print(f"Gradient Norm: {total_norm:.4f}")

In [70]:
import gc 

def check_cuda_memory():
    if not torch.cuda.is_available():
        return {"error": "CUDA is not available on this system."}

    device = torch.device("cuda")  # Assumes the first GPU (index 0) is used
    total_memory = torch.cuda.get_device_properties(device).total_memory
    allocated_memory = torch.cuda.memory_allocated(device)
    cached_memory = torch.cuda.memory_reserved(device)

    memory_info = {
        "Total Memory (MB)": total_memory / 1024**2,
        "Allocated Memory (MB)": allocated_memory / 1024**2,
        "Cached Memory (MB)": cached_memory / 1024**2,
        "Free Memory (MB)": (total_memory - cached_memory) / 1024**2,
    }

    print()
    for key, value in memory_info.items():
        print(f"{key}: {value:.2f} MB")
    print()

    return memory_info

def clear_cuda_cache():
    if not torch.cuda.is_available():
        return {"error": "CUDA is not available on this system."}

    gc.collect()
    torch.cuda.empty_cache()
    print("Cleaned cuda cache!")

clear_cuda_cache()
check_cuda_memory()

Cleaned cuda cache!

Total Memory (MB): 16269.25 MB
Allocated Memory (MB): 85.16 MB
Cached Memory (MB): 120.00 MB
Free Memory (MB): 16149.25 MB



{'Total Memory (MB)': 16269.25,
 'Allocated Memory (MB)': 85.16357421875,
 'Cached Memory (MB)': 120.0,
 'Free Memory (MB)': 16149.25}

In [71]:
# For demonstration purposes (fast training and bad performance),
# one can set smaller values:
# n_epochs = 20
# patience = 2
n_epochs = 180
patience = 180

l1_lambda = 1e-8

batch_size = 8 * 1024
epoch_size = math.ceil(len(train_idx) / batch_size)
best = {
    'val': -math.inf,
    'test': -math.inf,
    'epoch': -1,
}
# Early stopping: the training stops when
# there are more than `patience` consequtive bad updates.
# patience = 16
remaining_patience = patience

print('-' * 88 + '\n')
for epoch in range(n_epochs):
    loss_avg = 0.0
    loss_pure_avg = 0.0
    cnt_samples = 0

    cur_scheduler = scheduler # if epoch < 10 else scheduler

    for batch_idx in tqdm(
        torch.randperm(len(data['train']['y']), device=device).split(batch_size),
        desc=f'Epoch {epoch}',
        total=epoch_size,
    ):

        # print(batch_idx.shape)
        model.train()
        optimizer.zero_grad()
        Y_train_pred = apply_model(model, 'train', batch_idx)
        # print(Y_train_pred.shape, Y_train[batch_idx].shape)
        # print(Y_train_pred.dtype, Y_train[batch_idx].dtype)
        # Y_train_pred = Y_train_pred.mean(1)
        loss = loss_fn(Y_train_pred, Y_train[batch_idx].float())

        l1_penalty = 0
        for param in model.parameters():
            l1_penalty += torch.sum(torch.abs(param))
    
        loss_pure_avg += loss.detach()
        
        loss = loss + l1_lambda * l1_penalty

        loss_avg += loss.detach()
        cnt_samples += 1

        if grad_scaler is None:
            loss.backward()
            # print_grad_norms(model)
            optimizer.step()
        else:
            # print('hui')
            grad_scaler.scale(loss).backward()  # type: ignore
            grad_scaler.step(cur_optimizer)
            grad_scaler.update()

    cur_scheduler.step()
    print(f'AVG EPOCH LOSS: {loss_avg / cnt_samples}')
    print(f'AVG PURE EPOCH LOSS: {loss_pure_avg / cnt_samples}')
    print(f'LR: {scheduler.get_last_lr()[0]:.4f}')

    val_score = evaluate(model, 'val')
    test_score = evaluate(model, 'test')
    print(f'(val) {val_score:.4f} (test) {test_score:.4f}')

    if val_score > best['val']:
        print('🌸 New best epoch! 🌸')
        best = {'val': val_score, 'test': test_score, 'epoch': epoch}
        remaining_patience = patience
    else:
        remaining_patience -= 1

    if remaining_patience < 0:
        break

    print()
    if epoch % 10 == 0:
        print(evaluate(model, 'train'))

    # if epoch >= 2:
    #     swa_model.update_parameters(model)
    #     if epoch % 10 == 0:
    #         print('SWA MODEL EVALUATION:')
    #         print(evaluate(swa_model, 'train'))
    #         print(evaluate(swa_model, 'val'))
    #         print(evaluate(swa_model, 'test'))

    print()

print('\n\nResult:')
print(best)


# UPDATE BATCH NORM 
# swa_model.train()
# for batch_idx in tqdm(
#     torch.randperm(len(data['train']['y']), device=device).split(batch_size),
# ):
#     pred = model(data['train']['x_cont'][batch_idx], data['train']['x_cat'][batch_idx])



# Save the SWA model
torch.save(model.state_dict(), "model.pth")

----------------------------------------------------------------------------------------



Epoch 0: 100%|██████████| 19/19 [00:03<00:00,  5.68it/s]


AVG EPOCH LOSS: 0.03584987297654152
AVG PURE EPOCH LOSS: 0.03482028841972351
LR: 0.0020
mean prediction: 0.004557405784399649
(val) recall: 0.0521
(val) precision: 0.6154
(val) BEST threshold: 0.4167
(val) BEST f1: 0.4586
(val) BEST recall: 0.5136
(val) BEST precision: 0.4142
mean prediction: 0.0043121581825830875
(test) recall: 0.0411
(test) precision: 0.5366
(test) BEST threshold: 0.4167
(test) BEST f1: 0.4605
(test) BEST recall: 0.5145
(test) BEST precision: 0.4168
(val) 0.9472 (test) 0.9440
🌸 New best epoch! 🌸

mean prediction: 0.004226559152350901
(train) recall: 0.0472
(train) precision: 0.6221
(train) BEST threshold: 0.4167
(train) BEST f1: 0.4511
(train) BEST recall: 0.5038
(train) BEST precision: 0.4084
0.9453274122550738



Epoch 1: 100%|██████████| 19/19 [00:03<00:00,  5.67it/s]


AVG EPOCH LOSS: 0.025726275518536568
AVG PURE EPOCH LOSS: 0.02469715103507042
LR: 0.0020
mean prediction: 0.05714285714285714
(val) recall: 0.5060
(val) precision: 0.4765
(val) BEST threshold: 0.5000
(val) BEST f1: 0.4908
(val) BEST recall: 0.5060
(val) BEST precision: 0.4765
mean prediction: 0.06089608750525873
(test) recall: 0.4921
(test) precision: 0.4551
(test) BEST threshold: 0.4583
(test) BEST f1: 0.4779
(test) BEST recall: 0.5201
(test) BEST precision: 0.4421
(val) 0.9435 (test) 0.9382




...

AVG EPOCH LOSS: 0.018360700458288193
AVG PURE EPOCH LOSS: 0.01728203520178795
LR: 0.0011
mean prediction: 0.06152497808939527
(val) recall: 0.5179
(val) precision: 0.4530
(val) BEST threshold: 0.5000
(val) BEST f1: 0.4833
(val) BEST recall: 0.5179
(val) BEST precision: 0.4530
mean prediction: 0.06573411863693732
(test) recall: 0.5135
(test) precision: 0.4400
(test) BEST threshold: 0.5000
(test) BEST f1: 0.4739
(test) BEST recall: 0.5135
(test) BEST precision: 0.4400
(val) 0.9404 (test) 0.9358




Epoch 96: 100%|██████████| 19/19 [00:03<00:00,  5.68it/s]


AVG EPOCH LOSS: 0.018283382058143616
AVG PURE EPOCH LOSS: 0.017203722149133682
LR: 0.0011
mean prediction: 0.060297984224364595
(val) recall: 0.5147
(val) precision: 0.4593
(val) BEST threshold: 0.5000
(val) BEST f1: 0.4854
(val) BEST recall: 0.5147
(val) BEST precision: 0.4593
mean prediction: 0.06436684896928901
(test) recall: 0.5098
(test) precision: 0.4461
(test) BEST threshold: 0.5000
(test) BEST f1: 0.4758
(test) BEST recall: 0.5098
(test) BEST precision: 0.4461
(val) 0.9413 (test) 0.9367




Epoch 97:  95%|█████████▍| 18/19 [00:03<00:00,  5.66it/s]


KeyboardInterrupt: 

In [76]:
evaluate('val')

mean prediction: 0.08904469763365469
(val) recall: 0.5429
(val) precision: 0.3281
(val) BEST threshold: 0.5833
(val) BEST f1: 0.4327
(val) BEST recall: 0.4191
(val) BEST precision: 0.4473


0.9155711364300322

In [ ]:
a = torch.tensor([
    [1.0, 0.94, 0.8],
    [0.1, 0.6, 0.7],
    [0.23, 0.1, 0.9]
])

b = torch.tensor([1.0, 0.0, 1.0])

print(a.flatten(0, 1))
print(b.repeat_interleave(3))

In [72]:
weights_path = '/kaggle/working/sergey_tabm_model.pth'
torch.save(model.state_dict(), weights_path)

In [ ]:
model.predict()